# ATRX Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "ATRX"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to the cptac.algorithms get interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene

In [6]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

'''Show interacting protein list'''
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
NKTR
PNN
TP53
STAG2
H3F3B
SETDB1
NIPBL
PHIP
BPTF
RSF1
PRKDC
MUS81
ARID4A
RAD52
BDP1
XRCC6
SMC5
H3F3A
TPR
SCAF11
ATRX
DAXX
RAD51
MECP2
XRCC3
HIST2H3PS2
EIF4A2
EZH2
Mecp2
CBX5


### Test for significant comparisons in any of interacting proteins

In [7]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

                                 Comparison   P_Value
0  MECP2_transcriptomics_ENSG00000169057.18  0.000467





# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ATRX


Doing t-test comparisons

                                           Comparison       P_Value
0           TSPYL2_transcriptomics_ENSG00000184205.13  5.743268e-10
1     RP11-296E23.2_transcriptomics_ENSG00000265948.1  6.907699e-10
2           CCDC68_transcriptomics_ENSG00000166510.12  4.049959e-09
3      RP11-624L4.1_transcriptomics_ENSG00000259345.4  1.148094e-08
4           GREB1L_transcriptomics_ENSG00000141449.13  1.270396e-08
5      RP11-80F22.7_transcriptomics_ENSG00000260153.1  2.534988e-08
6           CDR1-AS_transcriptomics_ENSG00000281508.1  3.881078e-08
7     RP11-541N10.3_transcriptomics_ENSG00000260461.1  4.690104e-08
8        AC005237.4_transcriptomics_ENSG00000225521.1  4.760715e-08
9      RP11-279F6.1_transcriptomics_ENSG00000245750.6  8.828716e-08
10          EXOSC1_transcriptomics_ENSG00000171311.11  1.059106e-07
11       AC093585.6_transcriptomics_ENSG00000229996.1  1.130568e-07
12           PCAT29_transcriptomics_ENSG00000259641.4  1.398273e-07
13             DRG2_tr

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,ATRX,MECP2_transcriptomics_ENSG00000169057.18,True,4.671346e-04
0,Gbm,ATRX,TSPYL2_transcriptomics_ENSG00000184205.13,False,5.743268e-10
1,Gbm,ATRX,RP11-296E23.2_transcriptomics_ENSG00000265948.1,False,6.907699e-10
2,Gbm,ATRX,CCDC68_transcriptomics_ENSG00000166510.12,False,4.049959e-09
3,Gbm,ATRX,RP11-624L4.1_transcriptomics_ENSG00000259345.4,False,1.148094e-08
4,Gbm,ATRX,GREB1L_transcriptomics_ENSG00000141449.13,False,1.270396e-08
5,Gbm,ATRX,RP11-80F22.7_transcriptomics_ENSG00000260153.1,False,2.534988e-08
6,Gbm,ATRX,CDR1-AS_transcriptomics_ENSG00000281508.1,False,3.881078e-08
7,Gbm,ATRX,RP11-541N10.3_transcriptomics_ENSG00000260461.1,False,4.690104e-08
8,Gbm,ATRX,AC005237.4_transcriptomics_ENSG00000225521.1,False,4.760715e-08


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"